This code is a practice for the basic Regression Tree. 
This regression tree can deal with dataset with both discrete and continuous features, but requires that discrete value only takes two values,0 and 1.


In [35]:

# some data initialization. 
import numpy as np
import pandas as pd 
data= pd.read_excel('C:/Users/shufe/Dropbox/ds new/pwchkt.xlsx')
data.isnull().any(axis=1) # 判定每行会不会有空值的记录。（某条记录但凡在一个变量上有空值，就认为此记录
data=data.dropna(how='any')
data=data[['age','creditamount','guarantors','housing','foreigner','creditrating']]
data.loc[(data.creditrating == 2),'creditrating']=0
data_features=list(data.columns) 
data_features_original=data_features.copy() # This stores the all features. 
y='creditamount'
data_features.remove(y)
# we also need to create a dictionary indicating whether the feature is discrete or not
discrete={'age':'no','creditamount':'no','guarantors':'yes','housing':'yes','foreigner':'yes','creditrating':'yes'}
data_features

['age', 'guarantors', 'housing', 'foreigner', 'creditrating']

In [25]:
# this function choose the best feature given current data set. 
n_seq=8
def BestFeature(Dataset, unvisited_feature):
    # want to return the best feature and the resulting data split
    # but before this , you can already calculate the the mse of the current data
    orginal_mse=mse_cal(Dataset[y])
    threshold=[]
    mse=[]
    #print(unvisited_feature)
    for feature in unvisited_feature:
        if discrete[feature]=='yes': # this is discrete 
            # feature is name
            # feature level is number?
            # next, for each level in the feature_level, get
            mse.append(SplitDataSet(Dataset,feature, 0.5)[0])
            threshold.append(0.5)
        else: # the feature is continuous 
            # we first generate a threshold sequence 
            thres_seq=[   min(Dataset[feature])+i*(max(Dataset[feature])-min(Dataset[feature]))/(n_seq-1) 
                      for i in range(1,n_seq-1)   ]
            mse_seq= [ SplitDataSet(Dataset,feature, x)[0] for x in thres_seq]
            mse.append(min(mse_seq))
            threshold.append(thres_seq[mse_seq.index(min(mse_seq))])
        
    # we complete the loop with mse  series
    # we now return the best feature and the corresponding threshold 
    # for discrete, the corresponding threshold is always -1 
    best_feature=unvisited_feature[mse.index(min(mse))]
    best_threshold=threshold[mse.index(min(mse))]
    # we also return the splitted dat
    return best_feature,best_threshold
    # best_feature is a string. best_threshold is a value. 
    

In [17]:
# This function split the data into two parts given the feature and the threshold (value)
def SplitDataSet(Dataset,feature, value):
# we only consider the binary classification case
# given the dataset, feature and value, this function returns splitted data and the resulting MSE!
    Data_left=Dataset[Dataset[feature]<value]
    Data_right=Dataset[Dataset[feature]>=value]
# we want to calculate the mse of the data_left, data_right
    Total_mse=mse_cal(Data_left[y])+mse_cal(Data_right[y])
    return Total_mse,Data_left, Data_right 


In [20]:
import math
from statistics import mean 
def mse_cal(y_data):
    return(sum([pow((i-mean(y_data)),2) for i in y_data]))
    

In [26]:
# this recursive function creates a tree
def createTree(Dataset, layer,features):
    # we first need to check whether the layer has already reached the max
    if (layer==maxlayer  ):
        # you have already reached the deepest. all you have to do is to return the average y
        Tree=mean(Dataset[y])
    else:
        best_feature,best_threshold=BestFeature(Dataset, features)
        Tree = {}  # means that you want to generate a new son tree
        # the new son tree should contain the following features:
        Tree['spInd'] = best_feature # the feature you use to split the data
        featuresNext=features
        featuresNext.remove(best_feature)  # this feature would no longer be used in the next trees
        Tree['spVal'] = best_threshold # the threshold value 
        data_left, data_right = SplitDataSet(Dataset,best_feature, best_threshold)[1],SplitDataSet(Dataset,best_feature, best_threshold)[2]
        Tree['left'] = createTree(data_left,layer+1,featuresNext)
        Tree['right'] = createTree(data_right,layer+1,featuresNext)
    return Tree

In [36]:
maxlayer=3
createTree(data,1,data_features)

{'spInd': 'creditrating',
 'spVal': 0.5,
 'left': {'spInd': 'age',
  'spVal': 50.42857142857143,
  'left': 3765.631970260223,
  'right': 5434.935483870968},
 'right': {'spInd': 'foreigner',
  'spVal': 0.5,
  'left': 1932.909090909091,
  'right': 3037.5322338830583}}